# EC number from EXCEL file
We have information about the EC numbers so we may as well add it to the model.

In [1]:
from collections import defaultdict
from pathlib import Path

import cobra
import re
from datatable import dt, f, join, update

In [2]:
ROOT = Path.cwd().parent
model_file = str(ROOT / "iMENI452.xml")

In [3]:
model = cobra.io.read_sbml_model(model_file)

Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled


In [11]:
model_xls = dt.fread(ROOT / "M.nitroredencesGEM.tsv")

In [12]:
model_xls.head()

,Abbreviation,Description,Reaction,EC-Number,GPR,Gene,Subsystem,Reversible,Lower bound,Upper bound,Objective
,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪,▪▪▪▪,▪▪▪▪,▪
0,R00396,L-Alanine:NAD+ oxidoreductase (deaminating)_c0,ala-L[c] + nad[c] + h2o[c] <=> pyr[c] + nh4[c] + n…,EC-1.4.1.1,ANME2D_RS14405,ANME2D_RS14405,Alanine and Aspartate Metabolism,1,−1000,1000,0
1,ASPTA,aspartate transaminase,glu-L[c] + oaa[c] <=> akg[c] + asp-L[c],EC-2.6.1.1,ANME2D_RS08380 or ANME2D_RS05600 or ANME2D_RS07885…,ANME2D_RS08380 ANME2D_RS05600 ANME2D_RS07885 …,Alanine and Aspartate Metabolism,1,−1000,1000,0
2,ARGSS,argininosuccinate synthase,asp-L[c] + atp[c] + citr-L[c] <=> amp[c] + argsuc[…,EC-6.3.4.5,ANME2D_RS05565,ANME2D_RS05565,Alanine and Aspartate Metabolism,1,−1000,1000,0
3,ARGSL,argininosuccinate lyase,argsuc[c] <=> arg-L[c] + fum[c],EC-4.3.2.1,ANME2D_RS01360,ANME2D_RS01360,Alanine and Aspartate Metabolism,1,−1000,1000,0
4,ADSS,adenylosuccinate synthase,asp-L[c] + gtp[c] + imp[c] <=> dcamp[c] + gdp[c] +…,EC-6.3.4.4,ANME2D_RS03615,ANME2D_RS03615,Alanine and Aspartate Metabolism,1,−1000,1000,0
5,ADSL1,adenylosuccinate lyase,dcamp[c] <=> amp[c] + fum[c],EC-4.3.2.2,ANME2D_RS00280,ANME2D_RS00280,Alanine and Aspartate Metabolism,1,−1000,1000,0
6,R00490,L-Aspartate ammonia-lyase,asp-L[c] <=> nh4[c] + fum[c],EC-4.3.1.1,ANME2D_RS00300,ANME2D_RS00300,Alanine and Aspartate Metabolism,1,−1000,1000,0
7,ASPCT,aspartate carbamoyltransferase,asp-L[c] + cbp[c] <=> cbasp[c] + h[c] + pi[c],EC-2.1.3.2,ANME2D_RS01840 or ANME2D_RS01835,ANME2D_RS01840 ANME2D_RS01835,Alanine and Aspartate Metabolism,1,−1000,1000,0
8,ASP1DC,aspartate 1 decarboxylase,asp-L[c] + h[c] <=> ala-B[c] + co2[c],EC-4.1.1.11,,,Alanine and Aspartate Metabolism,1,−1000,1000,0


In [14]:
for reac in model.reactions:
    matched = model_xls[f.Abbreviation == reac.id, "EC-Number"]
    if matched.nrows:
        matched = matched[0, 0]
        if matched:
            reac.annotation["ec-code"] = matched

In [20]:
len({reac for reac in model.reactions if "ec-code" in reac.annotation})

789

In [19]:
len(model.reactions)

813

In [22]:
for reac in model.reactions:
    if "ec-code" in reac.annotation:
        if reac.annotation["ec-code"] is None:
            del reac.annotation["ec-code"]

In [24]:
len({reac for reac in model.reactions if "ec-code" in reac.annotation})

789

In [26]:
len({reac for reac in model.reactions if "ec-code" in reac.annotation if reac.annotation["ec-code"] != ""})

449

In [23]:
cobra.io.write_sbml_model(model, model_file)